In [ ]:
#| default_exp embed_to_tables

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export

from typing import Literal
from pathlib import Path
from copy import deepcopy

import pandas as pd
from pydantic_settings import CliApp

from clip_plot.configuration import Cfg
from clip_plot.images import ImageFactory
from clip_plot.embeddings import get_embeddings, write_embeddings

In [ ]:
#| export

_D = Cfg() # defaults

In [ ]:
#| export

def embed_to_tables_pipeline(
                images: list[Path] = deepcopy(_D.paths.images),
                embed_model: str = deepcopy(_D.model),
                output_dir: Path = deepcopy(_D.paths.output_dir),
                metadata: list[Path] | None = deepcopy(_D.paths.metadata),
                table_id: str = deepcopy(_D.paths.table_id),
                table_format: Literal["parquet", "csv"]=deepcopy(_D.paths.table_format),
                local_only: bool = False
                ):
                "Embed a folder of images and save embeddings as .npy file to disk"

                # using Path.cwd() to handle ../ names -- not sure if this is superstitious
                data_dir = Path.cwd() / Path(output_dir).resolve() / "data"

                imageEngine = ImageFactory(img_path=images,
                                           out_dir=data_dir.as_posix(),
                                           meta_dir=metadata)

                embeddings = get_embeddings(imageEngine, model_name=embed_model)

                def _model_shortname(n: str) -> str:
                        return "__".join(n.split("/")[-2:])

                embs_dir = data_dir/f"embeddings_{_model_shortname(embed_model)}"
                embs_dir.mkdir(parents=True, exist_ok=True)
                emb_paths = write_embeddings(embeddings, imageEngine.filenames, embs_dir)

                df = pd.DataFrame({"image_path": [p.as_posix() for p in imageEngine.image_paths],
                                   "image_filename": imageEngine.filenames,
                                   "embed_path": [str(e) for e in emb_paths]})

                if len(imageEngine.metadata) > 0:
                        df_meta = pd.DataFrame(imageEngine.metadata)
                        df_meta = df_meta.rename(columns={"filename": "image_filename"})
                        # drop "image_path" column if df_meta has it
                        if "image_path" in df_meta.columns:
                                df_meta = df_meta.drop(columns=["image_path"])

                        df = df.merge(df_meta.drop_duplicates(["image_filename"]), on="image_filename")

                ## standardize sort order of table
                # put standard columns first if they exist in df
                standard_cols = pd.Index(["image_path", "image_filename", "embed_path", "category", "tags", "x", "y"])
                cols_sorted = standard_cols.intersection(df.columns)
                # append non-standard columns, sorted alphabetically
                cols_sorted = cols_sorted.append(df.columns.difference(standard_cols).sort_values())
                df = df[cols_sorted]

                if table_format == "csv":
                        df.to_csv(data_dir / f"EmbedImages__{table_id}.csv", index=False)
                else:
                        df.to_parquet(data_dir / f"EmbedImages__{table_id}.parquet", index=False)

In [ ]:
#| export

def embed_to_tables_cli():
    """
    run embed_to_tables with CLI defined
    from pydantic-settings configuration object
    """
    cfg = CliApp.run(Cfg)
    embed_to_tables_pipeline(
        images=cfg.paths.images,
        embed_model=cfg.model,
        output_dir=cfg.paths.output_dir,
        metadata=cfg.paths.metadata,
        table_id=cfg.paths.table_id,
        table_format=cfg.paths.table_format
    )

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()